In [4]:
%matplotlib inline


In [5]:
from bifrost.fdmt import Fdmt
import numpy as np
import pylab as plt
import bifrost as bf

from astropy import units as u
import setigen as stg
import matplotlib.pyplot as plt

import hyperseti
import cupy as cp
import blimpy as bl

fdmt = Fdmt()

bshuf filter already loaded, skip it.


## Generate mock FRBs

In [6]:
def gauss2(x, a, x0, fwhm):
    """ Generate gaussian^2 pulse 
    
    Args:
        x: time series (np.arange())
        a: amplitude of pulse. (total integrated power across pulse)
        x0: index of peak value
        fwhm: width of signal in samples
    """
    sigma = (fwhm / 2) / np.sqrt(2*np.log(2))
    return a / np.sqrt(2*np.pi*sigma**2) * np.exp(-(x-x0)**2 / (2*sigma**2))


def generate_frb(frame, frb_params):
    """ Simple FRB generator 
    
    Args:
        frame (setigen.Frame): A filterbank frame of data, from setigen
        frb_params (dict): A dictionary with FRB parameters in it
        
    Notes:
        frb_params should contain width (in s), desired SNR, start time (in s), and DM
    """
    
    frb_width, frb_snr, frb_t0, frb_dm = frb_params['width'], frb_params['snr'], frb_params['t0'], frb_params['dm'], 

    frb_rms  = frame.get_intensity(snr=frb_snr)
    fch1 = frame.get_frequency(0)
    
    ## Generate pulse delays - array has same length as freqs
    width_in_chans = frb_width / frame.dt
    t0_in_samps = (frb_t0 / frame.dt) - frame.ts[0]
    tdel_in_samps = 4.15e-3 * frb_dm * ((fch1/1e9)**(-2) - (frame.fs/1e9)**(-2)) / frame.dt
    t0_in_samps = t0_in_samps + tdel_in_samps 
    
    # Time axis in samples
    t = np.arange(frame.tchans)

    # Use meshgrid to create (N_time, N_freq) array
    t2d, tdel2d = np.meshgrid(t, t0_in_samps)
    
    # Create pulse profile via 2D time (N_time,1) and time_delay (N_freq, 1) arrays
    profile = gauss2(t2d, frb_rms, tdel2d, width_in_chans)
    
    return profile.T

### Basic FRB models

In [7]:
frame = stg.Frame(fchans=1024*u.pixel,
                  tchans=1024*2*u.pixel,
                  df=100.0*u.kHz,
                  dt=1*u.ms,
                  fch1=300*u.MHz)

noise = frame.add_noise(x_mean=10, noise_type='chi2')

frb_params = {'snr': 10000.0, 
              't0': 500e-3, 
              'dm': 3, 
              'width': 10e-3}
p = generate_frb(frame, frb_params)
frame.data += p

frb_params = {'snr': 10000.0, 
              't0': 1000e-3, 
              'dm': 10, 
              'width': 10e-3}
p = generate_frb(frame, frb_params)
frame.data += p

frame.plot()

## Initialize FDMT

In [8]:
max_delay = frame.data.shape[0]

n_disp = max_delay
n_time = frame.data.shape[0]
n_chan = frame.data.shape[1]

fdmt.init(n_chan, n_disp, frame.fch1 / 1e6, frame.df / 1e6)

help(Fdmt().init)


Help on method init in module bifrost.fdmt:

init(nchan, max_delay, f0, df, exponent=-2.0, space='cuda') method of bifrost.fdmt.Fdmt instance



## Run FDMT

In [9]:
d_cpu = np.ascontiguousarray(np.expand_dims(frame.data.astype('float32').T, axis=0))

print(d_cpu.shape)

# Input shape is (1, n_freq, n_time)
d_in = bf.ndarray(d_cpu, dtype='f32', space='cuda')
d_out = bf.ndarray(np.zeros(shape=(1, n_disp, n_time)), dtype='f32', space='cuda')

print(d_in.shape, d_out.shape)
print(n_chan, n_time, n_disp, n_time)

fdmt.execute(d_in, d_out, negative_delays=True)

d_out = d_out.copy(space='system')

plt.figure(figsize=(10, 10))
plt.imshow(np.log(np.array(d_out)).squeeze(), aspect='auto')


ymax, xmax = np.unravel_index(np.argmax(d_out), (n_disp, n_time))
plt.scatter(xmax, ymax, marker='x', color='#cc0000')

plt.xlabel("Time")
plt.ylabel("Dispersion trial")
plt.colorbar()

(1, 1024, 2048)
(1, 1024, 2048) (1, 2048, 2048)
1024 2048 2048 2048


/tmp/ipykernel_27525/2974595870.py:17: RuntimeWarning: divide by zero encountered in log
  plt.imshow(np.log(np.array(d_out)).squeeze(), aspect='auto')


## Find maxima in output

In [7]:
img = cp.asarray(d_out.squeeze())
peaks, xvals, yvals = hyperseti.peak.prominent_peaks(img, num_peaks=3, min_xdistance=100, min_ydistance=100, threshold=10000)
xvals = cp.asnumpy(xvals)
yvals = cp.asnumpy(yvals)

In [8]:
plt.figure(figsize=(10, 10))
plt.imshow(np.log(np.array(d_out)).squeeze(), aspect='auto')
plt.scatter(xvals, yvals, marker='x', color='#cc0000')

/tmp/ipykernel_31284/3595156715.py:2: RuntimeWarning: divide by zero encountered in log
  plt.imshow(np.log(np.array(d_out)).squeeze(), aspect='auto')


In [8]:
plt.figure(figsize=(5, 5))
plt.imshow(np.log(np.array(d_out)).squeeze(), aspect='auto')
plt.scatter(xvals[0], yvals[0], marker='x', c='#cc0000')
plt.xlim(xvals[0] - 100, xvals[0] + 100)
plt.ylim(yvals[0] - 100, yvals[0] + 100)

plt.figure(figsize=(5, 5))
plt.imshow(np.log(np.array(d_out)).squeeze(), aspect='auto')
plt.scatter(xvals[1], yvals[1], marker='x', c='#cc0000')
plt.xlim(xvals[1] - 100, xvals[1] + 100)
plt.ylim(yvals[1] - 100, yvals[1] + 100)

/tmp/ipykernel_1494/3331985393.py:2: RuntimeWarning: divide by zero encountered in log
  plt.imshow(np.log(np.array(d_out)).squeeze(), aspect='auto')


NameError: name 'xvals' is not defined

## Filterbank data

In [25]:
import blimpy as bl

fp = 'i_00000.fil'
fil = bl.Waterfall(fp)

KeyError: 'signed'

In [30]:
max_delay = 128
gulp_size = 4 * 8192

def get_gulp(idx):
    d_cpu = np.ascontiguousarray(np.expand_dims(fil.data.squeeze().T, axis=0)[..., idx * gulp_size:(idx+1) * gulp_size])
    return d_cpu

d_cpu = get_gulp(12)

## Initialize FDMT
n_disp = max_delay
n_time = d_cpu.shape[2]
n_chan = d_cpu.shape[1]
fdmt.init(n_chan, n_disp, fil.header['fch1'], fil.header['foff'])

print(d_cpu.shape)

# Input shape is (1, n_freq, n_time)
d_in = bf.ndarray(d_cpu, dtype='f32', space='cuda')
d_out = bf.ndarray(np.zeros(shape=(1, n_disp, n_time)), dtype='f32', space='cuda')

print(d_in.shape, d_out.shape)
print(n_chan, n_time, n_disp, n_time)

# Execute FDMT
fdmt.execute(d_in, d_out, negative_delays=True)

d_out = d_out.copy(space='system')

NameError: name 'fil' is not defined

# plt.figure(figsize=(10, 6))
plt.imshow(np.log(np.array(d_out)).squeeze(), aspect='auto')

ymax, xmax = np.unravel_index(np.argmax(d_out), (n_disp, n_time))
plt.scatter(xmax, ymax, marker='x', color='#cc0000')

plt.xlim(xmax - 256, xmax+256)
plt.xlabel("Time")
plt.ylabel("Dispersion trial")
plt.colorbar()
print(DM[ymax])